Name:

Register Number:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import pickle
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pylab as plt

In [ ]:
customer_df = pd.read_csv('customers.csv')

An automobile company has plans to enter new markets with their existing products. After intensive market research, they’ve decided that the behavior of the new market is similar to their existing market.

In their existing market, the sales team has classified all customers into 4 segments (A, B, C, D ). Then, they performed segmented outreach and communication for a different segment of customers. This strategy has work exceptionally well for them. They plan to use the same strategy for the new markets.

You are required to help the manager to predict the right group of the new customers.

## Data Exploration

In [ ]:
customer_df.columns

In [ ]:
customer_df.dtypes

In [ ]:
customer_df.shape

In [ ]:
customer_df.isnull().sum()

In [ ]:
customer_df_cleaned = customer_df.dropna(axis=0)

In [ ]:
customer_df_cleaned.isnull().sum()

In [ ]:
customer_df_cleaned.shape

In [ ]:
customer_df_cleaned.dtypes

In [ ]:
customer_df_cleaned['Gender'].unique()

In [ ]:
customer_df_cleaned['Ever_Married'].unique()

In [ ]:
customer_df_cleaned['Graduated'].unique()

In [ ]:
customer_df_cleaned['Profession'].unique()

In [ ]:
customer_df_cleaned['Spending_Score'].unique()

In [ ]:
customer_df_cleaned['Var_1'].unique()

In [ ]:
customer_df_cleaned['Segmentation'].unique()

In [ ]:
categories_list=[['Male', 'Female'],
           ['No', 'Yes'],
           ['No', 'Yes'],
           ['Healthcare', 'Engineer', 'Lawyer', 'Artist', 'Doctor',
            'Homemaker', 'Entertainment', 'Marketing', 'Executive'],
           ['Low', 'Average', 'High']
           ]
enc = OrdinalEncoder(categories=categories_list)

In [ ]:
customers_1 = customer_df_cleaned.copy()

In [ ]:
customers_1[['Gender',
             'Ever_Married',
              'Graduated','Profession',
              'Spending_Score']] = enc.fit_transform(customers_1[['Gender',
                                                                 'Ever_Married',
                                                                 'Graduated','Profession',
                                                                 'Spending_Score']])

In [ ]:
customers_1.dtypes

In [ ]:
le = LabelEncoder()

In [ ]:
customers_1['Segmentation'] = le.fit_transform(customers_1['Segmentation'])

In [ ]:
customers_1.dtypes

In [ ]:
customers_1 = customers_1.drop('ID',axis=1)
customers_1 = customers_1.drop('Var_1',axis=1)

In [ ]:
customers_1.dtypes

In [ ]:
# Calculate the correlation matrix
corr = customers_1.corr()

# Plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        cmap="BuPu",
        annot= True)

In [ ]:
sns.pairplot(customers_1)

In [ ]:
sns.distplot(customers_1['Age'])

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(customers_1['Family_Size'])

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Family_Size',y='Age',data=customers_1)

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='Family_Size',y='Spending_Score',data=customers_1)

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='Family_Size',y='Age',data=customers_1)

In [ ]:
customers_1.describe()

In [ ]:
customers_1['Segmentation'].unique()

In [ ]:
X=customers_1[['Gender','Ever_Married','Age','Graduated','Profession','Work_Experience','Spending_Score','Family_Size']].values

In [ ]:
y1 = customers_1[['Segmentation']].values

In [ ]:
one_hot_enc = OneHotEncoder()

In [ ]:
one_hot_enc.fit(y1)

In [ ]:
y1.shape

In [ ]:
y = one_hot_enc.transform(y1).toarray()

In [ ]:
y.shape

In [ ]:
y1[0]

In [ ]:
y[0]

In [ ]:
X.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.33,
                                               random_state=50)

In [ ]:
X_train[0]

In [ ]:
X_train.shape

In [ ]:
scaler_age = MinMaxScaler()

In [ ]:
scaler_age.fit(X_train[:,2].reshape(-1,1))

In [ ]:
X_train_scaled = np.copy(X_train)
X_test_scaled = np.copy(X_test)

In [ ]:
# To scale the Age column
X_train_scaled[:,2] = scaler_age.transform(X_train[:,2].reshape(-1,1)).reshape(-1)
X_test_scaled[:,2] = scaler_age.transform(X_test[:,2].reshape(-1,1)).reshape(-1)

Name:

Register Number:

In [ ]:
# Creating the model
ai_brain = Sequential([
  # Develop your model here
])

In [ ]:
ai_brain.compile(optimizer='adam',
                 loss= # choose your loss function,
                 metrics=['accuracy'])

In [ ]:
#early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
ai_brain.fit(x=X_train_scaled,y=y_train,
             epochs= #choose the number of epochs,
             batch_size= # Choose the batchsize,
             validation_data=(X_test_scaled,y_test),
             )

In [ ]:
metrics = pd.DataFrame(ai_brain.history.history)

In [ ]:
metrics.head()

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
x_test_predictions = np.argmax(ai_brain.predict(X_test_scaled), axis=1)

In [ ]:
x_test_predictions.shape

In [ ]:
y_test_truevalue = np.argmax(y_test,axis=1)

In [ ]:
y_test_truevalue.shape

In [ ]:
print(confusion_matrix(y_test_truevalue,x_test_predictions))

In [ ]:
print(classification_report(y_test_truevalue,x_test_predictions))

In [ ]:
# Saving the Model
ai_brain.save('customer_classification_model.h5')

In [ ]:
# Saving the data
with open('customer_data.pickle', 'wb') as fh:
   pickle.dump([X_train_scaled,y_train,X_test_scaled,y_test,customers_1,customer_df_cleaned,scaler_age,enc,one_hot_enc,le], fh)

In [ ]:
# Loading the Model
ai_brain = load_model('customer_classification_model.h5')

In [ ]:
# Loading the data
with open('customer_data.pickle', 'rb') as fh:
   [X_train_scaled,y_train,X_test_scaled,y_test,customers_1,customer_df_cleaned,scaler_age,enc,one_hot_enc,le]=pickle.load(fh)

# Prediction for a single input

In [ ]:
x_single_prediction = np.argmax(ai_brain.predict(X_test_scaled[1:2,:]), axis=1)

In [ ]:
print(x_single_prediction)

In [ ]:
print(le.inverse_transform(x_single_prediction))